# 1. Setup, Library Installation, and Data Preparation
We have load the necessary libraries and segment the dataset to create our knowledge base of real jobs.

In [2]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu -qq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:


import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document


In [8]:
# 1. Load the raw dataset
df = pd.read_csv('/content/drive/MyDrive/Fake_Job_Posting_Detection/data/raw/fake_job_postings.csv')
df.fillna('', inplace=True)
df.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,,,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,0,1,0,,,,,,0


In [9]:
# 2. Setup Embedding Model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
embedding_function = SentenceTransformerEmbeddings(model_name=model_name)

print(f"Sentence Transformer Model ({model_name}) loaded.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-963596813.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name=model_name)


Sentence Transformer Model (all-MiniLM-L6-v2) loaded.


In [10]:
# 3. Create the Knowledge Base Source (Real Job Postings)
# Filter for verified Real Job Postings (fraudulent == 0)
real_jobs_df = df[df['fraudulent'] == 0].sample(n=300, random_state=42)
# Sampling 300 posts as requested for a focused knowledge base.

print(f"Knowledge Base Source created with {len(real_jobs_df)} verified real job postings.")

Knowledge Base Source created with 300 verified real job postings.


# 2. Build the Vector Store (FAISS)
We have embed the text of the real job postings and store them in FAISS for fast retrieval.

In [11]:
# Embed Real Postings and Build FAISS Vector Store

# Function to combine relevant text fields for the knowledge base
def combine_job_text(row):
    return (f"Title: {row['title']}. Company: {row['company_profile']}. "
            f"Description: {row['description']}. Requirements: {row['requirements']}")

# 1. Prepare Documents
real_job_documents = []
for index, row in real_jobs_df.iterrows():
    real_job_documents.append(
        Document(
            page_content=combine_job_text(row),
            metadata={"job_id": row['job_id'], "title": row['title']}
        )
    )



In [12]:
# 2. Create FAISS Vector Store
# This process embeds the text content of all 300 real jobs.
real_job_kb = FAISS.from_documents(real_job_documents, embedding_function)
print(f"FAISS Vector Store built with embeddings from {len(real_job_documents)} real job postings.")

FAISS Vector Store built with embeddings from 300 real job postings.


# 3. Implement the Advanced Retrieval Function
This function retrieves the most similar real job postings and uses them to explain why a suspicious post differs from the norm.

In [13]:
#  Retrieval and Generation Function (Similarity-Based Explanation)

def retrieve_and_explain_similarity(suspicious_job_text, real_job_kb, top_k=3):
    """Retrieves similar real jobs to contrast with the suspicious job."""

    # 1. Retrieval: Find top_k most similar (legitimate) job postings
    retrieved_docs = real_job_kb.similarity_search(suspicious_job_text, k=top_k)

    # 2. Construct the Prompt (Simulated LLM)
    context_list = []
    for i, doc in enumerate(retrieved_docs):
        # Limit content length for cleaner output
        content_snippet = doc.page_content[:150].replace('\n', ' ') + '...'
        context_list.append(f"Example {i+1} (Title: {doc.metadata['title']}): {content_snippet}")

    context_str = "\n".join(context_list)

    # 3. Generation (Simulated for this project)
    # The actual LLM prompt would ask for a contrastive explanation:

    simulated_explanation = (
        "**Conclusion: The job is suspicious because it deviates significantly "
        "from typical, real job postings for similar roles.**\n\n"
        "**Analysis based on top 3 closest legitimate examples:**\n"
        "The following real job postings were retrieved as highly similar to the suspicious posting:\n\n"
        f"{context_str}\n\n"
        "**Key Differences (LLM Explanation):**\n"
        "While the retrieved examples feature detailed company profiles, specific contact details, "
        "and balanced language, the suspicious posting is likely missing one or more of these elements. "
        "This type of explanation confirms the suspicious nature by showing it is an 'outlier' "
        "compared to verified postings."
    )

    return simulated_explanation

# 4. Demonstration with a New Posting
We have demonstrate this powerful RAG system using a known fake job.

In [14]:
#  RAG Demonstration (Using a Known Fake Job)

# Get a known fake job posting
fake_post = df[df['fraudulent'] == 1].iloc[0]

# Combine key text fields for the query to the vector store
job_text_query = combine_job_text(fake_post)

print(f"--- FAKE JOB POSTING (Actual Label: 1) ---")
print(f"Title: {fake_post['title']}\nDescription Snippet: {fake_post['description'][:200]}...")
print("\n" + "="*50 + "\n")

# Run Advanced RAG
explanation = retrieve_and_explain_similarity(job_text_query, real_job_kb)

print("--- RAG-GENERATED EXPLANATION (Similarity-Based) ---")
print(explanation)

--- FAKE JOB POSTING (Actual Label: 1) ---
Title: IC&E Technician
Description Snippet: IC&amp;E Technician | Bakersfield, CA Mt. PosoPrincipal Duties and Responsibilities: Calibrates, tests, maintains, troubleshoots, and installs all power plant instrumentation, control systems and elec...


--- RAG-GENERATED EXPLANATION (Similarity-Based) ---
**Conclusion: The job is suspicious because it deviates significantly from typical, real job postings for similar roles.**

**Analysis based on top 3 closest legitimate examples:**
The following real job postings were retrieved as highly similar to the suspicious posting:

Example 1 (Title: Project Engineer - Design Cost Engineer Exp - Atlanta North, GA): Title: Project Engineer - Design Cost Engineer Exp - Atlanta North, GA. Company: We Provide Full Time Permanent Positions for many medium to large US ...
Example 2 (Title: Regional Sales Manager): Title: Regional Sales Manager. Company: We Provide Full Time Permanent Positions for many medium to

# 5. pushing to github

In [17]:
%cd /content/drive/MyDrive/Fake_Job_Posting_Detection


/content/drive/MyDrive/Fake_Job_Posting_Detection


In [18]:
!ls -a


data  .git  .gitignore	models	notebooks  README.md  src


In [19]:
!git status


Refresh index: 100% (5/5), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/03_Model_Training_Evaluation.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/
	notebooks/04_RAG_Implementation.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [21]:
!ls -R


.:
data  models  notebooks  README.md  src

./data:
processed_data.csv  raw

./data/raw:
fake_job_postings.csv

./models:

./notebooks:
01_exploratry_data_analysis.ipynb  03_Model_Training_Evaluation.ipynb
02_preprocessing.ipynb		   04_RAG_Implementation.ipynb

./src:
